In [4]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout 

import pandas as pd
import numpy as np 

In [7]:
#Read Data 
train=pd.read_csv('Data/train.csv')
labels=train.ix[:,0].values.astype('int32')

X_train=(train.ix[:,1:].values).astype('float32')
X_test=(pd.read_csv('Data/test.csv').values).astype('float32')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#convert list of labels to binary class matrix 
y_train=np_utils.to_categorical(labels)

In [11]:
#Preprocessing : divide by max and subtract by mean
scale=np.max(X_train)
X_train/=scale
X_test/=scale

mean=np.std(X_train)
X_train-=mean
X_test-=mean

In [13]:
input_dim=X_train.shape[1]
nb_classes=y_train.shape[1]

In [16]:
#DDMLP
model=Sequential()

model.add(Dense(120,input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(120))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [25]:
#Use categorical cross entropy for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [26]:
model.fit(X_train,y_train, epochs=10, batch_size=16, validation_split=0.1, verbose=2)

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
11s - loss: 0.3883 - acc: 0.9230 - val_loss: 0.2425 - val_acc: 0.9476
Epoch 2/10
10s - loss: 0.4020 - acc: 0.9232 - val_loss: 0.2690 - val_acc: 0.9448
Epoch 3/10
11s - loss: 0.4016 - acc: 0.9230 - val_loss: 0.3325 - val_acc: 0.9419
Epoch 4/10
11s - loss: 0.4099 - acc: 0.9233 - val_loss: 0.3696 - val_acc: 0.9305
Epoch 5/10
10s - loss: 0.4035 - acc: 0.9269 - val_loss: 0.2966 - val_acc: 0.9460
Epoch 6/10
11s - loss: 0.4312 - acc: 0.9276 - val_loss: 0.4221 - val_acc: 0.9400
Epoch 7/10
10s - loss: 0.4122 - acc: 0.9279 - val_loss: 0.2519 - val_acc: 0.9486
Epoch 8/10
11s - loss: 0.4310 - acc: 0.9297 - val_loss: 0.3619 - val_acc: 0.9440
Epoch 9/10
11s - loss: 0.4228 - acc: 0.9285 - val_loss: 0.4152 - val_acc: 0.9426
Epoch 10/10
11s - loss: 0.4585 - acc: 0.9294 - val_loss: 0.4173 - val_acc: 0.9481


In [31]:
#Generating Test predictions 
preds=model.predict(X_test, verbose=0, batch_size=1)

In [35]:
def write_preds(preds, fname):
    pd.DataFrame({"ImageId":list(range(1,len(preds)+1)),"Label":preds}).to_csv(fname,index=False, header=True)

In [34]:
write_preds(preds,"keras-mlp.csv")

Exception: Data must be 1-dimensional